In [7]:
import scrapy
import requests
import urllib2
import BeautifulSoup
import pickle

In [2]:
def extract_title(text):
    pref = text[0:20]
    if "Pending" in pref:
        start = text.find('Pending')+7
    elif "Reviewed" in pref:
        start = text.find('Reviewed')+8
    elif "Indexed" in pref:
        start = text.find('Indexed')+7
    elif "Prelim" in pref:
        start = text.find('Indexed')+6
    elif "Thesis" in pref:
        start = text.find('Indexed')+6
    try:
        h = start
    except:
        print pref
        
    end = text.find('SFXButton(')
    return text[start:end].replace("\n", " ")

In [3]:
# get name, search for prof

# sample url: http://www.ams.org/mathscinet/search/publications.html?pg1=INDI&s1=304864&vfpref=html&r=1&extend=1
def find_papers(mid):
    prefix = "http://www.ams.org/mathscinet/search/publications.html?pg1=INDI&s1="
    suffix = "&vfpref=html&r=1&extend=1"
    url = prefix + mid + suffix
    
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup.BeautifulSoup(page)
    divList = soup.findAll('div', attrs={ "class" : "headlineText"})
    info_list = []
    for div in divList:
        num = div.find('a', attrs = {"class":"mrnum"})
        article_id = num.text
        link = num.get("href")

        authors = []
        for author_link in div.findAll('a'):
            href = author_link.get("href")
            if "author" in href:
                authors.append(href[40:])

        # article_title = div.find("span", attrs = {"class":"title"}).text
        article_title = extract_title(div.text)
        # article_title = div.find("span", attrs = {"class":"title"}).text

        info = {}
        
        info['article_id'] = article_id
        info['article_link'] = link
        info['article_title'] = article_title
        info['authors'] = authors
        # info['collaborators'] = []
        
        info_list.append(info)
    return info_list

In [4]:

# sample_url 

# http://www.ams.org/mathscinet/search/publications.html?amp=&loc=refcit&refcit=1085139&vfpref=html&r=1&extend=1
def find_citations(mid):
    prefix = "http://www.ams.org/mathscinet/search/publications.html?amp=&loc=refcit&refcit="
    suffix = "&vfpref=html&r=1&extend=1"
    
    
    remid = mid[2:]
    url = prefix + remid + suffix
    
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup.BeautifulSoup(page)
    divList = soup.findAll('div', attrs={ "class" : "headlineText"})
    info_list = []
    
    for div in divList:
        num = div.find('a', attrs = {"class":"mrnum"})
        article_id = num.text
        link = num.get("href")

        authors = []
        for author_link in div.findAll('a'):
            href = author_link.get("href")
            if "author" in href:
                authors.append(href[40:])

        # article_title = div.find("span", attrs = {"class":"title"}).text
        article_title = extract_title(div.text)
        
        info = {}
        
        info['article_id'] = article_id
        info['article_link'] = link
        info['article_title'] = article_title
        info['authors'] = authors
        
        info_list.append(info)

    return info_list

In [5]:
base = {}
aid_2_mid = {}
with open("ids2.txt") as f:    
    for m in f:
        sps = m.split("\t")
        id = sps[0]
        aid = sps[1]
        base[id] = aid
        aid_2_mid[aid] = id

# construct database for each professor

In [6]:
paper_base = {}

for aid in base.keys():
    if aid is not '0':
        paper_base[aid] = find_papers(base[aid])
    else:
        paper_base[aid] = []

In [9]:
paper_base

{'0': [],
 '1': [{'article_id': u'MR3414961',
   'article_link': u'/mathscinet/search/publdoc.html?extend=1&pg1=INDI&s1=604180&vfpref=html&r=1&mx-pid=3414961',
   'article_title': u'Andersen, J\xf8rgen Ellegaard;Chekhov, Leonid O.;Norbury, Paul;Penner, Robert C.Models of discretized moduli spaces, cohomological field theories, and  Gaussian means.J. Geom. Phys.98(2015),312\u2013339.81R10',
   'authors': [u'604180', u'47830', u'361773', u'137745']},
  {'article_id': u'MR3413187',
   'article_link': u'/mathscinet/search/publdoc.html?extend=1&pg1=INDI&s1=604180&vfpref=html&r=2&mx-pid=3413187',
   'article_title': u'Andersen, J\xf8rgen Ellegaard;J\xf8rgensen, S\xf8ren FugledeOn the Witten-Reshetikhin-Turaev invariants of torus bundles.J. Knot Theory Ramifications24(2015),no. 11,1550055, 48 pp.57R56 (53D50 57M25 57M27)',
   'authors': [u'604180', u'1130450']},
  {'article_id': u'MR3370620',
   'article_link': u'/mathscinet/search/publdoc.html?extend=1&pg1=INDI&s1=604180&vfpref=html&r=3&mx-p

In [8]:
with open("mathscinet_page_base.pickle", "wb") as f:
    pickle.dump(paper_base, f)

## Reverse the mapping
### - From paper to author

In [10]:
reverse_base = {}

for aid in paper_base.keys():
    papers = paper_base[aid]
    for p in papers:
        pid = p['article_id']
        reverse_base[pid] = p

In [12]:
for paper in reverse_base.values():
    paper['collaborators'] = []
    for au in paper['authors']:
        if au in aid_2_mid:
            paper['collaborators'].append(aid_2_mid[au])

In [16]:
real_papers = {}

for key in reverse_base.keys():
    paper = reverse_base[key]
    if len(paper['collaborators']) >=2:
        real_papers[key] = paper

In [32]:
"1975" in real_papers['MR0445014']['article_title']

True

### add year to it

In [36]:
def extract_year(text):
    for i in range(2011, 2017):
        if str(i) in text:
            return i
    return 0

In [37]:
for paper in real_papers.values():
    text = paper['article_title']
    paper['year'] = extract_year(text)

## Merge into old paper base

In [52]:
def id_maker(paper_set):
    ids = [int(p['id']) for p in paper_set]
    return max(ids)+1

In [60]:
import json
with open("papers.json") as f:
    old_paper_base = json.loads(f.read())

In [54]:
des_list = [p['description'] for p in old_paper_base['papers']] 

In [55]:
old_paper_base['papers']

[{u'collaborator_ids': [1, 99],
  u'date': u'2013',
  u'description': u'Andersen, J. E.; Penner, R. C.; Reidys, C. M.; Waterman, M. S. Topological classification and enumeration of RNA structures by genus. J. Math. Biol. 67 (2013), no. 5, 1261\u20131278.',
  u'id': u'1',
  u'publication': u'journal'},
 {u'collaborator_ids': [1, 99],
  u'date': u'2013',
  u'description': u'Andersen, J\xf8rgen E.; Chekhov, Leonid O.; Penner, R. C.; Reidys, Christian M.; Su\u0142kowski, Piotr Topological recursion for chord diagrams, RNA complexes, and cells in moduli spaces. Nuclear Phys. B 866 (2013), no. 3, 414\u2013443.',
  u'id': u'2',
  u'publication': u'journal'},
 {u'collaborator_ids': [1, 99],
  u'date': u'2012',
  u'description': u'Andersen, J\xf8rgen E.; Huang, Fenix W. D.; Penner, Robert C.; Reidys, Christian M. Topology of RNA-RNA interaction structures. J. Comput. Biol. 19 (2012), no. 7, 928\u2013943.',
  u'id': u'3',
  u'publication': u'journal'},
 {u'collaborator_ids': [2, 97],
  u'date': 

In [56]:
def if_dup(string_list, to_add):
    for string in string_list:
        if fuzz.ratio(string, to_add) > 95:
            return True
    return False

In [62]:
for paper in real_papers.values():
    dup_flag = if_dup(des_list, paper['article_title'])
    if paper['year'] == 0:
        continue
    elif dup_flag is True:
        continue
    else:
        
        add = {}
        add['date'] = paper['year']
        add['collaborator_ids'] = paper['collaborators']
        add['description'] = paper['article_title']
        add['id'] = id_maker(old_paper_base['papers'])
        
        old_paper_base['papers'].append(add)

In [63]:
len(old_paper_base['papers'])

460

In [21]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import math
import wikipedia
import Levenshtein
import difflib
import distance
import textblob
import numpy

In [22]:
def similarity_calculator(first, second):
    #if type(first) is not unicode:
    #    first = unicode(first, 'utf-8')
    #if type(second) is not unicode:
    #    second = unicode(second, 'utf-8')


    # 1. fuzzy matcher
    fr = fuzz.ratio(first, second)
    pr = fuzz.partial_ratio(first, second)
    sor = fuzz.token_sort_ratio(first, second)
    ser = fuzz.token_set_ratio(first, second)    
    
    # 2. sequence similarity
    s = difflib.SequenceMatcher(lambda x: x == " ", first, second)
    seq = round(s.ratio(), 3)
    
    # 3. edit distance
    # 3.1 absolute
    try:
        lv_ab = Levenshtein.distance(first, second)
    except:
        print "ooops", first, second
    
    # 3.2 jaro
    lv_ja = Levenshtein.jaro(first, second)
    
    # 3.3 jaro_winkler
    lv_jaw = Levenshtein.jaro_winkler(first, second)
    
    # 3.4 ratio
    lv_ra = Levenshtein.ratio(first, second)
    

    # 4 jarcard and 
    sr = distance.sorensen(first, second)
    ja = distance.jaccard(first, second)
    print fr, pr, sor, ser, seq, lv_ab, lv_ja, lv_jaw, lv_ra

In [25]:
fuzz.ratio('Andersen, J\xf8rgen E.; Chekhov, Leonid O.; Penner, R. C.; Reidys, Christian M.; Su\u0142kowski, Piotr Topological recursion for chord diagrams, RNA complexes, and cells in moduli spaces. Nuclear Phys. B 866 (2013), no. 3, 414\u2013443.', \
           'Ande J\xf8rgen E.; Chekhov, Leonid O Penner, R. C.; Reidys, Christian M.; Su\u0142kowski, Piotr Topological recursion for chord diagrams, RNAcomplexes,and cells in moduli spaces. Nuclear Phys. B 866 (2013), no. 3, 4142013443.')

98

# Computer number of paper they wrote together between y1 and y2

In [ ]:
def count_coop(gid1, gid2, y1, y2):
    val = 0
    aid1 = base[gid1]
    aid2 = base[gid2]
    # print aid1, aid2
    papers_1 = paper_base[aid1]
    # papers_2 = paper_base[aid2]
    
    for paper in papers_1:
        if paper['year'] >= y1 and paper['year']<= y2:
            if aid2 in paper['authors']:
                val += 1
    return val
    

In [ ]:
# 24
# 12

In [ ]:
count_coop("24","12",1000,2016)

In [ ]:
data = {}
for a1 in base.keys():
    data[a1] = {}
    for a2 in base.keys():
        if a1 is not a2:
            data[a1][a2] = count_coop(a1, a2, 2011, 2011)

In [ ]:
with open("input.csv", "w") as f:
    f.write("Source;Target;Weight;Type\n")
    for a1 in data.keys():
        for a2 in data[a1].keys():
            if data[a1][a2] > 0:
                f.write(a1)
                f.write(";")
                f.write(a2)
                f.write(";")
                f.write(str(data[a1][a2]))
                f.write(";")
                f.write("undirected\n")

In [ ]:
def print_single_year(year):
    data = {}
    for a1 in base.keys():
        data[a1] = {}
        for a2 in base.keys():
            if a1 is not a2:
                data[a1][a2] = count_coop(a1, a2, year, year)
                
    with open(str(year)+"_input.csv", "w") as f:
        f.write("Source;Target;Weight;Type\n")
        for a1 in data.keys():
            for a2 in data[a1].keys():
                if data[a1][a2] > 0:
                    f.write(a1)
                    f.write(";")
                    f.write(a2)
                    f.write(";")
                    f.write(str(data[a1][a2]))
                    f.write(";")
                    f.write("undirected\n")

In [ ]:
print_single_year(2011)
print_single_year(2012)
print_single_year(2013)
print_single_year(2014)
print_single_year(2015)
print_single_year(2016)

# Combine databases

In [ ]:
orig = "Boileau, Michel; Boyer, Steven; Cebanu, Radu; Walsh, Genevieve S. Knot commensurability and the Berge conjecture. Geom. Topol. 16 (2012), no. 2, 625–664."
new = 'Knot commensurability and the Berge conjecture.'

In [ ]:
orig = "Steven B. Bradlow"
new = "Bradlow, S"

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import math
import wikipedia
import Levenshtein
import difflib
import distance
import textblob
import numpy

In [ ]:
import json
with open("papers.json") as f:
    old_base = json.loads(f.read())

In [ ]:
old_base.keys()

In [ ]:
paper_base

In [ ]:
# make a mapping from 
mapping = {}
for key in base.keys():
    mapping[base[key]] = key
    
mapping

In [ ]:
# go through paper_base, filter only 2011+ papers

out = []
for key in paper_base.keys():
    paper_list = paper_base[key]
    for paper in paper_list:
        if paper['year'] >= 2011:
            if check_if_coop(paper['authors'], mapping):
                newone = {}
                newone['collaborator_ids'] = get_ids(paper['authors'], mapping)
                newone['date'] = str(paper['year'])
                newone['description'] = paper['article_title']
                out.append(newone)

In [ ]:
out_base = []
for unit in out:
    f = True
    for exi in out_base:
        if unit['description'] == exi['description']:
            f = False
    if f:
        out_base.append(unit)
        

In [ ]:
len(out_base)

In [ ]:
out_base

In [ ]:
def check_if_coop(authors, mapping):
    val = 0
    for au in authors:
        if au in mapping.keys():
            val += 1
    if val>=2:
        return True
    else:
        return False
    
def get_ids(authors, mapping):
    ret = []
    for au in authors:
        if au in mapping.keys():
            ret.append(int(mapping[au]))
    return ret

In [ ]:
# out is auto database
# old_base is old database

final = []

for a in old_base['papers']:
    f = True
    for b in final:
        if title_compare(a, b) is True:
            f = False
    if f:
        final.append(a)
        
for a in out_base:
    f = True
    for b in final:
        if title_compare(a, b) is True:
            f = False
    if f:
        final.append(a)
        
        

In [ ]:
len(old_base['papers'])

In [ ]:
len(final)

In [ ]:
def title_compare(a, b):
    return fuzz.partial_ratio(a['description'], b['description'])>95

In [ ]:
def ids_compare(a, b):
    return set(a['collaborator_ids']) == set(b['collaborator_ids'])

In [ ]:
def date_compare(a, b):
    return a['date'] == b['date']

In [ ]:
with open("prof.txt", "r") as f:
    profile = json.loads(f.read())

In [ ]:
for prof in profile['items']:
    id = prof['member_id']
    try:
        prof[u'mathsci_id'] = mapping[id]
    except:
        prof[u'mathsci_id'] = 0

In [ ]:
profile

In [ ]:
import json
with open('new_profile.json', 'w') as outfile:
    json.dump(profile, outfile)

In [ ]:
import json
with open('new_papers.json', 'w') as outfile:
    json.dump(final, outfile)